In [21]:
# Imports

import sys
import time
import string
import json
sys.path.append('../../')
# General
import numpy as np
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from contextlib import redirect_stdout
# brain2brain
from brain2brain import utils
from brain2brain import generators
from brain2brain import tcn_experiments
# TCN
from brain2brain.tcn import TCN
from brain2brain.tcn import compiled_tcn
# TF

from tensorflow.keras import backend as K, Model, Input, optimizers
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras import layers
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras import Input, Model
from tensorflow.keras.layers import Dense, Flatten, GRU, Lambda, TimeDistributed, Activation
# import wandb

In [33]:
%pwd

'/home/tmarcu/brain2brain/experiments/notebooks'

In [42]:
with open("../experiment_params/tcn_experiment_m2m_2.json") as f:
    experiment_dict = json.load(f)

In [43]:
path = experiment_dict['path']
batch_size = experiment_dict['batch_size']
epochs = experiment_dict['epochs']
lookback_window = experiment_dict['lookback_window']
length_pred = experiment_dict['length_pred']
delay_pred = experiment_dict['delay_pred']
samples_per_second = experiment_dict['samples_per_second']
electrode_selection=experiment_dict['electrode_selection']
debug_mode = experiment_dict['debug_mode']
num_feat = experiment_dict['num_feat']
num_classes = experiment_dict['num_classes']
nb_filters = experiment_dict['nb_filters']
kernel_size = experiment_dict['kernel_size']
dilations=experiment_dict['dilations']
nb_stacks = experiment_dict['nb_stacks']
output_len = experiment_dict['output_len']
padding = experiment_dict['padding']
use_skip_connections = experiment_dict['use_skip_connections']
return_sequences = experiment_dict['return_sequences']
regression = experiment_dict['regression']
dropout_rate = experiment_dict['dropout_rate']
name = experiment_dict['name']
kernel_initializer = experiment_dict['kernel_initializer']
activation = experiment_dict['activation']
opt = experiment_dict['opt']
lr = experiment_dict['lr']
# Read saved paths for training.
saved_paths = utils.get_file_paths(path)
# Split the train files into a training and validation set.
train, val = utils.split_file_paths(saved_paths, 0.8)
total_electrode_count = utils.get_file_shape(train[0])[1]
# Electrodes
electrode_count = len(electrode_selection)
# Sampling of electrodes.
timesteps_per_sample = int(lookback_window // samples_per_second)
# Training Generator

(36853, 114)


In [ ]:
train_stes

In [49]:
# Training Generator
train_generator = generators.FGenerator(file_paths=train,
                                        lookback=lookback_window, length=length_pred, delay=delay_pred,
                                        batch_size=batch_size, sample_period=samples_per_second,
                                        electrodes=electrode_selection, shuffle=True, debug=debug_mode)
# Validation Generator
val_generator = generators.FGenerator(file_paths=val,
                                      lookback=lookback_window, length=length_pred, delay=delay_pred,
                                      batch_size=batch_size, sample_period=samples_per_second,
                                      electrodes=electrode_selection, shuffle=False, debug=debug_mode)
train_steps = len(train_generator)
val_steps = len(val_generator)


In [50]:
model = load_model("/home/tmarcu/brain2brain/experiment_logs/tcn_experiment_m2m_2/model.h5",
                   custom_objects={"TCN": TCN})

(None, 128, 5)


In [51]:
predictions = model.predict_generator(val_generator, steps=val_steps,
                            callbacks=None, max_queue_size=10, workers=1,
                            use_multiprocessing=True, verbose=1)
predictions_path = "predictions.json"
np.save(predictions_path, predictions)

102/102 [==============================] - 193s 2s/step


In [52]:
predictions.shape

(104448, 1, 5)

In [53]:

predictions[0]

array([[-0.04569653, -0.06815685, -0.06594924, -0.04025666, -0.02818307]],
      dtype=float32)

In [59]:
val_generator[0][1][0]

array([[-0.82527386, -0.27210556,  0.03552958,  0.23143432, -0.36599276]])